In [14]:
import tensorflow as tf
print(tf.__version__)

2.0.0-rc0


In [15]:
tf.test.is_gpu_available()


True

In [16]:
import tensorflow as tf
#gpu = tf.config.experimental.list_physical_devices(device_type='GPU')
#tf.config.experimental.set_memory_growth(gpu[0], True)

# Dense
class MyDense(tf.keras.layers.Layer):
    def __init__(self, units=32, **kwargs):
        self.units = units
        super(MyDense, self).__init__(**kwargs)
    
    # build方法一般定义Layer需要被训练的参数
    def build(self, input_shape):
        self.w = self.add_weight(shape=(input_shape[-1], self.units), initializer='random_normal', trainable=True, name='w')
        self.b = self.add_weight(shape=(self.units,),initializer='random_normal', trainable=True, name='b')
        super(MyDense,self).build(input_shape) # 相当于设置self.built = True
    
    # call方法一般定义正向传播运算逻辑，__call__方法调用了它
    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b
    
    # 如果要让自定义的Layer通过Functional API 组合成模型时可以序列化，需要自定义get_config方法
    def get_config(self):
        config = super(MyDense, self).get_config()
        config.update({'units': self.units})
        return config

In [17]:
from sklearn import datasets
iris = datasets.load_iris()
data = iris.data
labels = iris.target

In [18]:
data[:5]

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2]])

In [19]:
labels #(150,3)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [20]:
# 网络 函数式构建的网络
inputs = tf.keras.Input(shape=(4,))
x = MyDense(units=16)(inputs)
x = tf.nn.tanh(x)
x = MyDense(units=3)(x)
x = tf.keras.layers.Dense(16)(x)
predictions = tf.nn.softmax(x)
model = tf.keras.Model(inputs=inputs,outputs=predictions)

In [21]:
import numpy as np

In [22]:
data = np.concatenate((data,labels.reshape(150,1)),axis=-1)
np.random.shuffle(data)

In [23]:
labels = data[:,-1]
data = data[:,:4]

In [24]:
# 优化器 Adam
# 损失函数 交叉熵损失函数
# 评估函数 acc

# 编码运行过程中遇到了ERROR:Function call stack:distributed_function
# 从网上找了各种解决方法都不可以，网上的方法大约是明确指明使用GPU，后来解决的办法是在运行中的其它 notebook实例SHUT DOWN就可以了
model.compile(optimizer=tf.keras.optimizers.Adam(),
             loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
             metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

# keras train
model.fit(data, labels, batch_size=32, epochs=100, shuffle=True)

Train on 150 samples
Epoch 1/100
150/150 [==============================] - 0s 1ms/sample - loss: 2.7722 - sparse_categorical_accuracy: 0.0000e+00
Epoch 2/100
150/150 [==============================] - 0s 80us/sample - loss: 2.7709 - sparse_categorical_accuracy: 0.2533
Epoch 3/100
150/150 [==============================] - 0s 66us/sample - loss: 2.7696 - sparse_categorical_accuracy: 0.3333
Epoch 4/100
150/150 [==============================] - 0s 60us/sample - loss: 2.7682 - sparse_categorical_accuracy: 0.3333
Epoch 5/100
150/150 [==============================] - 0s 66us/sample - loss: 2.7666 - sparse_categorical_accuracy: 0.3333
Epoch 6/100
150/150 [==============================] - 0s 66us/sample - loss: 2.7648 - sparse_categorical_accuracy: 0.3333
Epoch 7/100
150/150 [==============================] - 0s 160us/sample - loss: 2.7627 - sparse_categorical_accuracy: 0.3333
Epoch 8/100
150/150 [==============================] - 0s 73us/sample - loss: 2.7603 - sparse_categorical_accuracy

In [25]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 4)]               0         
_________________________________________________________________
my_dense_2 (MyDense)         (None, 16)                80        
_________________________________________________________________
tf_op_layer_Tanh_1 (TensorFl [(None, 16)]              0         
_________________________________________________________________
my_dense_3 (MyDense)         (None, 3)                 51        
_________________________________________________________________
dense_1 (Dense)              (None, 16)                64        
_________________________________________________________________
tf_op_layer_Softmax_1 (Tenso [(None, 16)]              0         
Total params: 195
Trainable params: 195
Non-trainable params: 0
_____________________________________________________________

In [26]:
model.save('keras_model_tf_version.h5')

In [27]:
_custom_objects = {
    "MyDense" : MyDense,
}

In [28]:
new_model = tf.keras.models.load_model("keras_model_tf_version.h5", custom_objects=_custom_objects)

In [29]:
y_pred = new_model.predict(data)

In [30]:
np.argmax(y_pred, axis=1)

array([1, 2, 1, 2, 0, 2, 1, 0, 0, 2, 2, 1, 0, 0, 2, 2, 2, 1, 0, 1, 0, 0,
       1, 2, 2, 2, 1, 0, 0, 2, 2, 1, 1, 2, 1, 2, 0, 2, 1, 2, 2, 0, 2, 1,
       2, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 2, 1, 1, 0, 1, 1, 1, 2,
       2, 2, 2, 1, 0, 2, 2, 0, 2, 0, 0, 1, 1, 2, 1, 1, 2, 0, 1, 2, 2, 1,
       1, 1, 0, 2, 1, 0, 1, 2, 2, 1, 1, 0, 2, 2, 2, 0, 0, 0, 1, 2, 1, 0,
       2, 0, 2, 0, 2, 1, 2, 0, 2, 0, 1, 2, 2, 0, 0, 2, 0, 2, 0, 0, 0, 0,
       1, 2, 0, 1, 1, 0, 0, 2, 1, 1, 2, 0, 1, 0, 2, 0, 2, 2], dtype=int64)

In [31]:
labels

array([1., 2., 1., 2., 0., 2., 1., 0., 0., 1., 2., 1., 0., 0., 2., 2., 2.,
       1., 0., 1., 0., 0., 1., 2., 1., 2., 1., 0., 0., 2., 2., 1., 1., 2.,
       1., 2., 0., 2., 1., 1., 2., 0., 2., 1., 2., 1., 1., 1., 0., 0., 0.,
       0., 0., 0., 1., 1., 0., 0., 2., 1., 1., 0., 1., 1., 1., 2., 2., 2.,
       2., 1., 0., 2., 2., 0., 2., 0., 0., 1., 1., 2., 1., 1., 2., 0., 1.,
       2., 2., 1., 1., 1., 0., 1., 1., 0., 1., 2., 2., 1., 1., 0., 2., 2.,
       2., 0., 0., 0., 1., 2., 1., 0., 2., 0., 2., 0., 2., 1., 2., 0., 2.,
       0., 1., 2., 2., 0., 0., 2., 0., 2., 0., 0., 0., 0., 1., 2., 0., 1.,
       1., 0., 0., 2., 1., 1., 2., 0., 1., 0., 2., 0., 2., 2.])